# 第四章：迭代器与生成器

In [1]:
#4.1 手动遍历迭代器
#问题：你想遍历一个可迭代对象中的所有元素，但是却不想使用for循环
#解决方案：为了手动遍历可迭代对象，使用next函数并在代码中捕获STOPITERATION异常，比如

In [8]:
def manual_iter():
    with open('somefile.txt') as f:
        try:
            while True:
                line = next(f)
                print(line, end='')
        except StopIteration:
            pass

In [10]:
help(next)

Help on built-in function next in module builtins:

next(...)
    next(iterator[, default])
    
    Return the next item from the iterator. If default is given and the iterator
    is exhausted, it is returned instead of raising StopIteration.



In [9]:
with open('somefile.txt') as f:
    while True:
        line = next(f)
        if line is None:
            break
        print(line, end='')

=== Keeping the Last N Items

==== Problem

You want to keep a limited history of the last few items seen
during iteration or during some other kind of processing.

==== Solution

Keeping a limited history is a perfect use for a `collections.deque`.
For example, the following code performs a simple text match on a
sequence of lines and prints the matching line along with the previous
N lines of context when found:

[source,python]
----
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            for pline in previous_lines:
                print(lline, end='')
            print(line, end='')
            print()
        previous_lines.append(line)

# Example use on a file
if __name__ == '__main__':
    with open('somefile.txt') as f:
         search(f, 'python', 5)
----

==== Discussion

Using `deque(maxlen=N)` creates a fixed size queue.  When new items
are added and the q

StopIteration: 

大多数情况下，我们会使用for循环语句用来遍历一个可迭代对象，但是，偶尔也需要对迭代做
更加精准的控制，这时候理解底层迭代机制就显得尤为重要了

In [12]:
items = [1, 2, 3]
# Get the iterator
it = iter(items)
# run the iterator
next(it), next(it), next(it)

(1, 2, 3)

In [13]:
next(it)

StopIteration: 

## 4.2 代理迭代

问题：你构建了一个自定义容器，里面包含有列表，元组和其他可迭代对象，你想直接在这个
新容器上执行迭代操作

In [14]:
#解决方案：实际上你有只需要一个__iter__()方法，讲迭代操作代理容器内部的对象上去

In [ ]:
class Node:
    def __init__(self, value):
        self._value = value
        self.children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value) # 将对象格式化转换成repr格式，计算机能看懂的格式
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
if __name__ == "__main__":
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    # Outputs Node(1), Node(2)
    for ch in root:
        print(ch)
#__iter__方法只是简单的将迭代请求？传递给内部的_children属性
#__repr__,__iter__是怎么起作用的

In [17]:
print(repr('hello')), print('hello')

'hello'
hello


(None, None)

In [22]:
#Python的迭代器协议需要__iter__方法返回一个实现了__next__方法的迭代器对象，如果你只是#迭代请求
#遍历其他容器的内容，你无需担心底层是怎样实现的，你所要做的只是传递迭代请求即可

In [21]:
#这里的iter函数的使用简化了代码，iter只是简单的通过调用s.__iter__()方法来返回对应的迭代器对象
#就跟len(s)会调用s.__len()__原理是一样的

## 4.3 使用生成器创建新的迭代模式

问题：你想实现一个自定义迭代模式，跟普通的内置函数range，reversed函数不一样

解决方案：如果你想实现一种新的迭代模式，使用一个生成器函数来定义它
下面是一个生产某个范围内浮点数的生成器

In [26]:
def frange(start, stop, increment):
    x = start 
    while x < stop:
        yield x
        x += increment
#为了使用这个函数，你可以用for循环迭代它或者使用其他接受一个可迭代对象的函数，例如，
#sum， list等

In [24]:
for n in frange(0,5,0.5):
    print(n)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5


In [25]:
list(frange(0, 1, 0.123))

[0, 0.123, 0.246, 0.369, 0.492, 0.615, 0.738, 0.861, 0.984]

一个函数中需要有一个yield语句即可将其转换为一个生成器，跟普通函数不同的是，生成器
只能用于迭代操作，下面是一个实验，向你展示函数底层的工作机制

In [27]:
def countdown(n):
    print('Starting to count from', n)
    while n > 0:
        yield n
        n -= 1
    print("Done!!!")

In [28]:
c = countdown(5)

In [29]:
c

<generator object countdown at 0x00000198CCA8EB88>

In [30]:
next(c)

Starting to count from 5


5

In [31]:
next(c),next(c)

(4, 3)

In [32]:
next(c), next(c)

(2, 1)

In [33]:
next(c)

Done!!!


StopIteration: 

一个生成器函数主要特征是它只会回应在迭代中使用到的next操作，一旦生成器函数返回退出，
迭代终止，我们在迭代中通常使用for语句会自动处理这些细节，所以无需担心

4.4 实现迭代器协议

问题：你想构建一个能支持迭代操作的自定义对象，并希望找到一个能实现迭代协议的简单方法

解决方案：目前为止，在一个对象上实现迭代最简单的方式是使用一个生成器函数，在4.2小节中
使用Node类来表示属性数据结构，你可能想实现一个以深度优先方式遍历树形节点的生成器

In [35]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self._children)
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first() # ?
            
if __name__ == "__main__":
    root = Node(0)
    child1= Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
    
    for ch in root.depth_first():
        print(ch)

#depth first方法简单直观，它首先返回自己本身并迭代每一个子节点并通过调用子节点的depth first
#方法使用yield from语句返回对应元素

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In [36]:
#python的迭代协议要求一个__iter__方法返回一个特殊的迭代器对象，这个迭代器对象实现__next__方法
#并通过StopIteration异常表示迭代完成，但是实现这些通常会比较繁琐，下面使用一个关联迭代器
#重新实现

In [39]:
class Node2:
    def __init__(self, value):
        self._value = value
        self._children = []
        
    def __repr__(self):
        return 'Node({!r})'.format(selfl._value)
    
    def add_child(self, node):
        self._children.append(node)
        
    def __iter__(self):
        return iter(self,_children)
    
    def depth_first(self):
        return DepthFirstIterator(self)
    
class DepthFirstIterator(object):
    """
    """
    def __init__(self, start_node):
        self._node = start_node
        self._children_iter = None
        self._child_iter = None
        
    def __iter__(self):
        return self
    
    def __next__(self):
        # Return myself if just started; create an iterator for children
        if self._children_iter is None:
            self._children_iter = iter(self._node)
            return self._node
        #If processing a child, return its next item
        elif self._children_iter:
            try:
                nextchild = next(self._children_iter)
                return nextchild
            except StopIteration:
                self._children_iter = None
                return next(self)
        #advance to the next child and start its iteration
        else:
            self._children_iter = next(self._children_iter).depth_first()
            return next(self)
#DepthFirstIterator类和上面使用的生成器的版本工作原理类似，但是写起来繁琐
#因为迭代器必须在迭代处理过程中维护大量的状态信息，代码晦涩难懂，讲你的迭代器定义
#为一个生成器后一切迎刃而解

## 4.5 反向迭代

问题：你想反方向迭代一个序列

解决方案：使用内置的reversed函数

In [41]:
a = list('12345')
for x in reversed(a):
    print(x)

5
4
3
2
1


In [43]:
#反向迭代仅仅当对象的大小可预先确定或者对象实现__reversed__的特殊方法时才能生效，如果
#两者都不符合，那你必须先将对象转换为一个列表才行
#！如果可迭代对象元素很多的话，将其预先转换为一个列表要消耗大量的内存

In [50]:
#可以在定义类上实现__reversed__()方法来实现反向迭代

class Countdown:
    def __init__(self, start):
        self.start = start
        
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
            
    # Reversed iterator
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n 
            n += 1

for rr in Countdown(10):
    print(rr)
    
for rr in reversed(Countdown(10)):
    print(rr)

#定义一个反向迭代器可以使得代码非常的高效，不需要先将数据填充到一个列表中然后再去
#反向迭代这个列表

10
9
8
7
6
5
4
3
2
1
1
2
3
4
5
6
7
8
9
10


4.6 带有外部状态的生成器函数

问题：你想顶一个生成器函数，但是它会调用某个你想暴露给用户使用的外部状态值

解决方案：如果你想让你的生成器暴露外部状态给用户，别忘了你可以简单的将它实现一个类
然后把生成器函数放到iter方法中获取，比如

In [67]:
from collections import deque

class linehistory:
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
        
    def __iter__(self):
        for lineno, line in enumerate(self.lines, 1):
            self.history.append((lineno, line))
            yield line
            
    def clear(self):
        self.history.clear()
        
#为了使用这个类，你可以将它当做一个普通的生成器函数，然而，由于可以创建一个实例对象
#于是你可以访问内部属性值，比如，history属性或者clear方法，代码示例如下

In [68]:
with open('somefile.txt') as f:
    lines = linehistory(f)
    for line in lines:
        if 'python' in line:
            for lineno, hline in lines.history:
                print('{}:{}'.format(lineno, hline), end='')

13:N lines of context when found:
14:
15:[source,python]
30:if __name__ == '__main__':
31:    with open('somefile.txt') as f:
32:         search(f, 'python', 5)


In [69]:
#关于生成器，很容易掉进函数无所不能的陷阱。 
#如果生成器函数需要跟你的程序其他部 分打交道的话(比如暴露属性值，允许通过方法调用来控制等等)
#可能会导致你的代码异常的复杂。 如果是这种情况的话，可以考虑使用上面介绍的定义类的方式。 
#在 __iter__() 方法中定义你的生成器不会改变你任何的算法逻辑。 
#由于它是类的一部分， 所以允许你定义各种属性和方法来供用户使用。 
#一个需要注意的小地方是，如果你在迭代操作时不使用for循环语句，那么你得先调用 iter() 函数。
#比如：

In [70]:
f = open('somefile.txt')
lines = linehistory(f)
next(lines)

TypeError: 'linehistory' object is not an iterator

In [71]:
it = iter(lines)
next(it), next(it)

('=== Keeping the Last N Items\n', '\n')

4.7 迭代器切片

问题：你想得到一个由迭代器生成的切片对象，但是标准切片操作并不能做到

解决方案：itertoois.islice()正好适用于在迭代器和生成器上做切片操作，比如

In [72]:
def count(n):
    while True:
        yield n
        n += 1
        
c = count(0)
c[10:20]

TypeError: 'generator' object is not subscriptable

In [73]:
import itertools

for x in itertools.islice(c, 10, 20):
    print(x)

10
11
12
13
14
15
16
17
18
19


In [74]:
#迭代器和生成器不能使用标准的切片操作，因为它们的长度事先我们并不知道(并且也没 有实现索引)。 
#函数 islice() 返回一个可以生成指定元素的迭代器，它通过遍历并丢弃 直到切片开始索引位置的所有元素。 
#然后才开始一个个的返回元素，并直到切片结束索 引位置。 
#这里要着重强调的一点是 islice() 会消耗掉传入的迭代器中的数据。 必须考虑到迭代器 是不可逆的这个事实。 
#所以如果你需要之后再次访问这个迭代器的话，那你就得先将它 里面的数据放入一个列表中